In [1]:
import numpy as np
import sklearn.datasets as dataset
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state


t0 = time.time()
train_samples = 5000

X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

In [3]:
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

In [4]:
import hilbert 
def flattenImg(img):
        size = img.size
        x, y = img.shape
        order = int(np.log2(x))
        d = [0]*size
        hilbert_curve = hilbert.HilbertCurve(order, 2)
        for r in range(x):
            for c in range(y):
                idx = hilbert_curve.distance_from_coordinates([r,c])
                d[idx] = img[r,c]
        return d
def flatten2Img(flatten, shape):
    x, y = shape
    dd = []
    img = np.zeros(shape)
    order = int(np.log2(x))
    hilbert_curve = hilbert.HilbertCurve(order, 2)
    for d_idx, v in enumerate(flatten):
        x, y = hilbert_curve.coordinates_from_distance(d_idx)
        img[x,y] = v
    return img
def transformImg(sample):
    sSample = resize(sample.reshape(28,28),(16,16), True)
    sFilter = np.array([1 if x > 0 else 0 for x in sSample.flatten()]).reshape(16,16)
    return flattenImg(sFilter)

In [5]:
# !pip install tqdm
from tqdm import tqdm

In [6]:
transformX = []
for s in tqdm(X):
    transformX.append(transformImg(s))

  0%|          | 0/70000 [00:00<?, ?it/s]/home/huynhnguyen/.local/lib/python2.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/huynhnguyen/.local/lib/python2.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
100%|██████████| 70000/70000 [03:52<00:00, 301.01it/s]


In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [8]:
sample = transformX[0]
def build_doc(sample):
    doc = []
    for i,x in enumerate(sample):
        left = i-8 if i-8>=0 else 0
        right = i + 8 if i + 8 < len(sample) else len(sample)
        ngram = sample[left:right]
        str_sample = ''.join([str(w) for w in ngram])
        doc.append(str_sample)
    return doc

In [ ]:
sentences = []
for sample in tqdm(transformX):
    sentences.append(build_doc(sample))

 49%|████▉     | 34492/70000 [01:14<01:16, 462.50it/s]

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
model = Word2Vec(min_count=1)
model.build_vocab(sentences)

In [11]:
str(model)

'Word2Vec(vocab=29177, size=100, alpha=0.025)'

In [49]:
model.train(sentences, total_examples=model.corpus_count, epochs=10)
def check_performance(target_idx, topk=10):
    score = cosine_similarity([sent_vecs[target_idx]], sent_vecs[0:1000])[0]
    print(sorted(score)[::-1][:topk])
    print(y[target_idx], [y[idx] for idx in np.argsort(score)][::-1][:topk])
check_performance(0)
check_performance(2)
check_performance(1)
check_performance(3)
sample = sentences[0]
def sentence_vec(sample):
    return np.mean(np.vstack([model.wv[s] for s in sample]),0)
sent_vecs = []
for sample in tqdm(sentences):
    sent_vecs.append(sentence_vec(sample))

2019-02-18 17:01:06,222 : WARNING : Effective 'alpha' higher than previous training cycles
2019-02-18 17:01:06,224 : INFO : training model with 3 workers on 29177 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-02-18 17:01:07,231 : INFO : EPOCH 1 - PROGRESS: at 17.05% examples, 1867180 words/s, in_qsize 5, out_qsize 0
2019-02-18 17:01:08,239 : INFO : EPOCH 1 - PROGRESS: at 34.38% examples, 1875854 words/s, in_qsize 5, out_qsize 2
2019-02-18 17:01:09,239 : INFO : EPOCH 1 - PROGRESS: at 53.21% examples, 1941943 words/s, in_qsize 5, out_qsize 0
2019-02-18 17:01:10,241 : INFO : EPOCH 1 - PROGRESS: at 71.98% examples, 1971510 words/s, in_qsize 6, out_qsize 0
2019-02-18 17:01:11,246 : INFO : EPOCH 1 - PROGRESS: at 91.20% examples, 1996836 words/s, in_qsize 6, out_qsize 1
2019-02-18 17:01:11,713 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-18 17:01:11,723 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02

2019-02-18 17:01:54,743 : INFO : EPOCH 8 - PROGRESS: at 92.71% examples, 1692104 words/s, in_qsize 5, out_qsize 0
2019-02-18 17:01:55,239 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-02-18 17:01:55,245 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-02-18 17:01:55,246 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-02-18 17:01:55,247 : INFO : EPOCH - 8 : training on 17920000 raw words (10978873 effective words) took 6.5s, 1683720 effective words/s
2019-02-18 17:01:56,255 : INFO : EPOCH 9 - PROGRESS: at 15.27% examples, 1669636 words/s, in_qsize 5, out_qsize 0
2019-02-18 17:01:57,255 : INFO : EPOCH 9 - PROGRESS: at 31.09% examples, 1700835 words/s, in_qsize 6, out_qsize 0
2019-02-18 17:01:58,257 : INFO : EPOCH 9 - PROGRESS: at 47.13% examples, 1720954 words/s, in_qsize 5, out_qsize 0
2019-02-18 17:01:59,258 : INFO : EPOCH 9 - PROGRESS: at 63.01% examples, 1727374 words/s, in_qsize 6, out_qsize 0
2019-02-18 17:02:

[1.0000002, 0.9875598, 0.9872982, 0.98714143, 0.9862945, 0.98567253, 0.9854299, 0.98518646, 0.98507315, 0.98490345]
('0', ['0', '0', '5', '0', '2', '0', '2', '5', '5', '2'])
[1.0, 0.99876714, 0.9983008, 0.99827737, 0.9982349, 0.99814105, 0.99814105, 0.99812186, 0.99795115, 0.99784446]
('1', ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1'])
[0.99999994, 0.9957791, 0.9955552, 0.9953248, 0.9952843, 0.9951736, 0.99511194, 0.99508834, 0.9949149, 0.9947593]
('4', ['4', '5', '1', '1', '2', '5', '1', '8', '1', '4'])
[1.0, 0.98728466, 0.9856657, 0.9847254, 0.98434, 0.98372805, 0.9831263, 0.9827213, 0.9820826, 0.9814534]
('2', ['2', '3', '5', '0', '3', '0', '0', '4', '2', '4'])


100%|██████████| 70000/70000 [00:55<00:00, 1263.20it/s]


In [54]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
docmodel = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4)

2019-02-18 17:34:21,545 : INFO : collecting all words and their counts
2019-02-18 17:34:21,546 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-02-18 17:34:21,778 : INFO : PROGRESS: at example #10000, processed 2560000 words (11095125/s), 19236 word types, 10000 tags
2019-02-18 17:34:22,009 : INFO : PROGRESS: at example #20000, processed 5120000 words (11121533/s), 22866 word types, 20000 tags
2019-02-18 17:34:22,255 : INFO : PROGRESS: at example #30000, processed 7680000 words (10481747/s), 24923 word types, 30000 tags
2019-02-18 17:34:22,514 : INFO : PROGRESS: at example #40000, processed 10240000 words (9957100/s), 26419 word types, 40000 tags
2019-02-18 17:34:22,749 : INFO : PROGRESS: at example #50000, processed 12800000 words (10919634/s), 27426 word types, 50000 tags
2019-02-18 17:34:22,985 : INFO : PROGRESS: at example #60000, processed 15360000 words (10946016/s), 28423 word types, 60000 tags
2019-02-18 17:34:23,224 : INFO : collected 29177 

2019-02-18 17:35:00,282 : INFO : EPOCH - 5 : training on 17920000 raw words (11049362 effective words) took 7.2s, 1535308 effective words/s
2019-02-18 17:35:00,283 : INFO : training on a 89600000 raw words (55246904 effective words) took 35.6s, 1551787 effective words/s


In [60]:
docmodel.vocabulary.raw_vocab()

TypeError: 'collections.defaultdict' object is not callable

In [48]:
from IPython.display import HTML

input_form = """
<div id="sketch">
  <canvas id="paint"></canvas>
</div>

<div id="settings"> Colours
  <button onclick="getColor('blue');">Blue</button>
  <button onclick="getColor('red');">Red</button>
  <button onclick="getColor('green');">Green</button>
  <button onclick="getColor('white');">White</button>
  <button onclick="getColor('#333333');">Eraser</button>
</div>

<div id="settings"> Brush Size
  <button onclick="getSize('2');">Small</button>
  <button onclick="getSize('5');">Med</button>
  <button onclick="getSize('10');">Large</button>
  <button onclick="getSize('20');">X Large</button>
</div>
"""

javascript = """
<script type="text/Javascript">
    var canvas = document.getElementById('paint');
    var ctx = canvas.getContext('2d');

    var sketch = document.getElementById('sketch');
    var sketch_style = getComputedStyle(sketch);
    canvas.width = 32;
    canvas.height = 32;

    var mouse = {x: 0, y: 0};

    /* Mouse Capturing Work */
    canvas.addEventListener('mousemove', function(e) {
      mouse.x = e.pageX - this.offsetLeft;
      mouse.y = e.pageY - this.offsetTop;
    }, false);

    /* Drawing on Paint App */
    ctx.lineJoin = 'round';
    ctx.lineCap = 'round';

    ctx.strokeStyle = "red";
    function getColor(colour){ctx.strokeStyle = colour;}

    function getSize(size){ctx.lineWidth = size;}


    //ctx.strokeStyle = 
    //ctx.strokeStyle = document.settings.colour[1].value;

    canvas.addEventListener('mousedown', function(e) {
        ctx.beginPath();
        ctx.moveTo(mouse.x, mouse.y);

        canvas.addEventListener('mousemove', onPaint, false);
    }, false);

    canvas.addEventListener('mouseup', function() {
        canvas.removeEventListener('mousemove', onPaint, false);
    }, false);

    var onPaint = function() {
        ctx.lineTo(mouse.x, mouse.y);
        ctx.stroke();
    };
</script>
"""
css = """
<style>
*{font-family:sans-serif;}

#settings { display: block; text-align:center}


#paint {border: 1px solid black; background : #333333; margin-left: auto; margin-right: auto; display: block;}
</style>
"""
HTML(input_form + javascript + css)